In [6]:
from pyMez import *

Importing pyMez, this should take roughly 30 seconds
Importing Code.DataHandlers.GeneralModels
It took 0.765 s to import Code.DataHandlers.GeneralModels
Importing Code.DataHandlers.HTMLModels
It took 0.211 s to import Code.DataHandlers.HTMLModels
Importing Code.DataHandlers.NISTModels
It took 2.335 s to import Code.DataHandlers.NISTModels
Importing Code.DataHandlers.TouchstoneModels
It took 0.015 s to import Code.DataHandlers.TouchstoneModels
Importing Code.DataHandlers.XMLModels
It took 0.193 s to import Code.DataHandlers.XMLModels
Importing Code.DataHandlers.ZipModels
It took 0.011 s to import Code.DataHandlers.ZipModels
Importing Code.InstrumentControl.Experiments
It took 0.736 s to import Code.InstrumentControl.Experiments
Importing Code.InstrumentControl.Instruments
It took 0.054 s to import Code.InstrumentControl.Instruments
Importing Code.Utils.Names
It took 0.044 s to import Code.Utils.Names
It took 4.364 s to import all of the active modules


In [1]:
import visa

In [2]:
rm=visa.ResourceManager()

In [ ]:
rm.list_resources()

In [3]:
scope=rm.open_resource("GPIB0::7::INSTR")

In [4]:
scope.query("*IDN?")

u'Agilent Technologies,86100C,MY47440684,A.08.00\n'

In [25]:
class HighSpeedOscope(VisaInstrument):
    """Control Class for high speed oscilloscopes. Based on code from Diogo """
    def initialize(self,**options):
        """Initializes the oscilloscope  for data collection"""
        defaults = {"reset": True}
        initialize_options = {}
        for key, value in defaults.iteritems():
            initialize_options[key] = value
        for key, value in options.iteritems():
            initialize_options[key] = value
        pass

    def measure_waves(self,**options):
        """Returns data for a measurement in an AsciiDataTable"""
        defaults = {"number_frames": 1,"number_points":self.get_number_points(),
                    "timebase_scale":self.get_timebase_scale(),"channels":[1,2,3,4],
                    "initial_time_offset":self.get_time_position(),"timeout_measurement":10000,
                    "save_data":False,"data_format":"dat","directory":os.getcwd(),
                    "specific_descriptor":"Scope","general_descriptor":"Measurement","add_header":False,
                    "output_table_options":{"data_delimiter":"  ","treat_header_as_comment":True}
                    }
        self.measure_options = {}
        for key, value in defaults.iteritems():
            self.measure_options[key] = value
        for key, value in options.iteritems():
            self.measure_options[key] = value

        self.set_number_points(self.measure_options["number_points"])
        channel_string_list=["CHAN1","CHAN2","CHAN3","CHAN4"]

        # begin by setting timeout to timeout_measurement
        timeout=self.resource.timeout
        self.resource.timeout=self.measure_options["timeout_measurement"]

        # now calculate timestep
        self.measure_options["timestep"]=self.measure_options["timebase_scale"]/self.measure_options["number_points"]
        time_step=self.measure_options["timestep"]

        # now configure the scope for data transfer
        # define the way the data is transmitted from the instrument to the PC
        # Word -> 16bit signed integer
        #self.write(':WAV:FORM WORD')
        self.write(':WAV:FORM ASCII')
        # little-endian
        #self.write(':WAV:BYT LSBF')
        number_rows=self.measure_options["number_points"]*self.measure_options["number_frames"]
        # this is the way diogo did it
        # number of points
        number_points = self.measure_options["number_points"]
        frames_data = []
        for frame_index in range(self.measure_options["number_frames"]):
            new_frame=[]

            # calculate time position for this frame
            time_position=frame_index*self.measure_options["timebase_scale"]+self.measure_options["initial_time_offset"]

            # define postion to start the acquisition
            self.write(':TIM:POS {0}ns'.format(time_position))

            # acquire channels desired
            channel_string=""
            for channel_index,channel in enumerate(self.measure_options["channels"]):
                if channel_index==len(self.measure_options["channels"])-1:
                    channel_string=channel_string+"{0}".format(channel_string_list[channel-1])
                else:
                    channel_string = channel_string + "{0},".format(channel_string_list[channel - 1])
            channel_command=":DIG {0}".format(channel_string)
            self.write(channel_command)

            # trigger reading and wait
            self.write("*OPC?")

            # get data from the necessary channels
            for channel_read_index,channel_read in enumerate(self.measure_options["channels"]):
                # get data for channel 1
                self.write(':WAV:SOUR CHAN{0}'.format(channel_read))
                # get data
                data_column=self.resource.query(':WAV:DATA?')
                data_column=data_column.replace("\n","").split(",")
                new_frame.append(data_column)
                #print("{0} is {1}".format("data_column",data_column))
            frames_data.append(new_frame)
            
        # reshape measurement data
        measurement_data=[range(len(frames_data[0])) for x in range(number_points*len(frames_data))]
#         print(len(measurement_data))
#         print(len(measurement_data[0]))
#         print("{0} is{1}".format("len(frames_data)",len(frames_data)))
#         print("{0} is{1}".format("len(frames_data[0])",len(frames_data[0])))
#         print("{0} is{1}".format("len(frames_data[0][0])",len(frames_data[0][0])))

        for frame_index,frame in enumerate(frames_data):
            for column_index,column in enumerate(frame):
                for row_index,row in enumerate(column):
                    number_rows=len(column)
                    #print("{0} is {1}".format("([row_index+frame_index*number_rows],[column_index],[frame_index])",
                                               ([row_index+frame_index*number_rows],[column_index],[frame_index])))
                    measurement_data[row_index+frame_index*number_rows][column_index]=frames_data[frame_index][column_index][row_index]
        # reset timeout
        self.resource.timeout=timeout
        data_out=[]
        time_start=self.measure_options["initial_time_offset"]

        for row_index,data_row in enumerate(measurement_data):
            new_row=[time_start+row_index*time_step]+data_row
            data_out.append(new_row)
        if self.measure_options["add_header"]:
            header=[]
            for key,value in self.measure_options.iteritems():
                header.append("{0} = {1}".format(key,value))
        else:
            header=None
        column_names=["Time"]
        for channel in self.measure_options["channels"]:
            column_names.append(channel_string_list[channel-1])

        table_options={"data":data_out,
                       "header":header,
                       "specific_descriptor":self.measure_options["specific_descriptor"],
                       "general_descriptor":self.measure_options["general_descriptor"],
                       "extension":"dat",
                       "directory":self.measure_options["directory"],"column_names":column_names}
        for key,value in self.measure_options["output_table_options"].iteritems():
            table_options[key]=value
        output_table=AsciiDataTable(None,**table_options)
        if self.measure_options["save_data"]:

            data_save_path=auto_name(specific_descriptor=self.measure_options["specific_descriptor"],
                                     general_descriptor=self.measure_options["general_descriptor"],
                                     directory=self.measure_options["directory"],
                                     extension='dat'
                                     ,padding=3)
            output_table.path=data_save_path
            output_table.save()
        return output_table




    def get_error_state(self):
        """Returns the error state of the oscope"""
        state=self.query(':SYSTEM:ERROR? STRING')
        self.error_state=state
        return state

    def set_number_points(self,number_points=16384):
        """Sets the number of points for the acquisition"""
        self.write(':ACQ:POINTS {0}'.format(number_points))

    def get_number_points(self):
        """Returns the number of points in the waveform"""
        number_points= int(self.query(':ACQ:POINTS?'))
        return number_points

    def set_time_position(self,time=50):
        """Sets the time in ns to start the acquisition"""
        self.write(':TIM:POS {0}ns'.format(time))

    def get_time_position(self):
        """Returns the time position in ns"""
        position=float(self.query(":TIM:POS?"))
        return position*10**9

    def set_timebase_scale(self,time_scale=40.96):
        """Sets the timebase scale in ns"""
        self.write(':TIM:SCAL {0}ns'.format(time_scale))

    def get_timebase_scale(self):
        """Returns the timebase scale in ns"""
        time_scale=float(self.query(':TIM:SCAL?'))
        return time_scale*10**9

    def set_trigger_source(self,source="FPAN"):
        """Sets the tigger source, 'FPAN' Frontpanel or 'FRUN' freerun"""
        self.write(':TRIG:SOUR {0}'.format(source))

    def get_trigger_source(self):
        """Returns the trigger source FPAN for FrontPanel or FRUN for free run"""
        source=self.query(':TRIG:SOUR?')

    def set_trigger_level(self,level=10):
        """Sets the trigger level in mv"""
        self.write(':TRIG:LEV {0}m'.format(level))

    def get_trigger_level(self):
        """Returns the trigger level"""
        level=self.query(':TRIG:LEV?')

    def set_channel_scale(self,scale=10,channel=1):
        """Sets the scale in mv of channel. Default is 10mv/division on channel 1"""
        self.write(':CHAN{0}:SCAL {1}m'.format(channel,scale))

    def get_channel_scale(self,channel=1):
        "Returns the scale for a specified channel, the default is channel 1"
        scale=self.query(':CHAN{0}:SCAL?'.format(channel))
        return scale

    def set_channel_bandwidth(self,bandwidth="LOW",channel=1):
        """Sets the specified channel's bandwith to LOW, MED or HIGH, default is to set channel 1 to LOW"""
        self.write(':CHAN{0}:BAND {1}'.format(channel,bandwidth))

    def get_channel_bandwidth(self,channel=1):
        """Returns the selected channels bandwidth"""
        bandwidth=self.query(':CHAN{0}:BAND?'.format(channel))
        return bandwidth

    def set_trigger_slope(self,slope="POS"):
        """Sets the trigger slope on the oscope choose from POS or NEG"""
        self.write(':TRIG:SLOP {0}'.format(slope))

    def get_trigger_slope(self):
        """Returns the trigger slope either POS or NEG"""
        slope=self.query(":TRIG:SLOP?")
        return slope


In [26]:
oscope=HighSpeedOscope("GPIB0::7")

The information sheet was not found defaulting to address


In [9]:
oscope.get_error_state()

u'36,\n'

In [10]:
oscope.get_timebase_scale()

0.1

In [11]:
print oscope.get_number_points()
oscope.set_number_points(number_points=300)
print oscope.get_number_points()

1350
300


In [22]:
oscope.get_time_position()

50.0

In [186]:
table=oscope.measure_waves(timeout_measurement=10000,number_frames=12)

([row_index+frame_index*number_rows],[column_index],[frame_index]) is ([0], [0], [0])
([row_index+frame_index*number_rows],[column_index],[frame_index]) is ([1], [0], [0])
([row_index+frame_index*number_rows],[column_index],[frame_index]) is ([2], [0], [0])
([row_index+frame_index*number_rows],[column_index],[frame_index]) is ([3], [0], [0])
([row_index+frame_index*number_rows],[column_index],[frame_index]) is ([4], [0], [0])
([row_index+frame_index*number_rows],[column_index],[frame_index]) is ([5], [0], [0])
([row_index+frame_index*number_rows],[column_index],[frame_index]) is ([6], [0], [0])
([row_index+frame_index*number_rows],[column_index],[frame_index]) is ([7], [0], [0])
([row_index+frame_index*number_rows],[column_index],[frame_index]) is ([8], [0], [0])
([row_index+frame_index*number_rows],[column_index],[frame_index]) is ([9], [0], [0])
([row_index+frame_index*number_rows],[column_index],[frame_index]) is ([10], [0], [0])
([row_index+frame_index*number_rows],[column_index],[

In [ ]:
oscope.resource.timeout

In [13]:
print len(table.data[1])

5


In [34]:
print len(table.data)

101


In [39]:
%matplotlib wx
plt.plot(table["CHAN2"])
plt.show()

In [40]:
from pyMez.Code.Analysis.Fitting import *

In [91]:
sine=FunctionalModel(equation=".2*sin(omega*x+phi)+c",parameters="omega phi c",variables="x")

In [93]:
print sine(x=0,phi=1,omega=2,c=0)

0.168294196962


In [206]:
sine.fit_data(x_data=np.array(map(lambda x:float(x),table["Time"])),y_data=np.array(map(lambda x:float(x),table["CHAN3"])),initial_guess={ 'c': 0, 'omega':6./4.* 140, 'phi': 0})

In [207]:
(sine.parameter_values["phi"]%np.pi)*180/np.pi

71.29810339014506

In [190]:
x_data=np.array(map(lambda x:float(x),table["Time"]))
y_data=np.array(map(lambda x:float(x),table["CHAN2"]))
plt.plot(x_data,y_data)
plt.plot(x_data,sine(x_data),label="Fit")
plt.legend()
plt.show()

In [42]:
plt.plot(table["Time"],table["CHAN2"])
plt.show()

In [82]:
vna=VNA("GPIB::20")

In [100]:
vna.write("SOUR1:POW:STAT ON")

(19L, <StatusCode.success: 0>)

In [95]:
import math
math.sqrt(.05)

0.22360679774997896

In [99]:
vna.write("SOUR1:FREQ{0} {1}".format(1,10**9))

(24L, <StatusCode.success: 0>)

In [96]:
math.sqrt(50*100*10**-6)

0.07071067811865475

In [112]:
vna.set_power(-10)

In [105]:
vna.get_power()

u'0\n'

In [106]:
pm=NRPPowerMeter("GPIB::14")

In [107]:
pm.initialize()

In [108]:
pm.get_reading()

2.709479e-07

In [109]:
vna.initialize()

In [16]:
scope.write(':WAV:FORM ASCII')

(17L, <StatusCode.success: 0>)

In [ ]:
scope

In [111]:
out=oscope.measure_waves(channels=[3,4])

VisaIOError: VI_ERROR_TMO (-1073807339): Timeout expired before operation completed.

In [20]:
oscope.write(':WAV:SOUR CHAN{0}'.format(2))
                # get data
data_column=oscope.resource.query(':WAV:DATA?')

In [ ]:
scope.

In [21]:
data_column

u'-7.27E-003,-5.144E-002,2.091E-002,-4.787E-002,-6.563E-002,1.971E-002,3.636E-002,-5.588E-002,4.888E-002,-1.528E-002,4.412E-002,-6.563E-002,5.248E-002,-3.540E-002,-2.53E-003,-8.46E-003,-1.883E-002,3.65E-003,-9.05E-003,5.993E-002,-2.476E-002,4.739E-002,-4.044E-002,-6.269E-002,-6.97E-003,6.023E-002,4.440E-002,-3.216E-002,9.9E-004,1.614E-002,-7.541E-002,3.783E-002,4.711E-002,-5.321E-002,-5.560E-002,5.367E-002,-4.787E-002,-2.329E-002,-3.837E-002,5.456E-002,-4.03E-003,2.952E-002,-9.35E-003,6.01E-003,-5.50E-003,7.339E-002,-5.202E-002,3.994E-002,2.894E-002,1.941E-002,-3.748E-002,-6.208E-002,-1.37E-003,-3.540E-002,-5.648E-002,-4.579E-002,6.023E-002,-4.787E-002,7.729E-002,6.682E-002,8.601E-002,3.994E-002,-4.430E-002,-6.181E-002,-2.919E-002,-2.772E-002,-4.92E-003,3.221E-002,8.119E-002,5.42E-003,5.697E-002,3.07E-003,5.038E-002,5.755E-002,-1.348E-002,-1.852E-002,3.667E-002,4.949E-002,2.537E-002,-1.645E-002,1.138E-002,7.100E-002,3.636E-002,-6.39E-003,-7.58E-003,-5.618E-002,-1.825E-002,2.983E-002,-5

In [107]:
table.options["directory"]=r'C:\Share\LongDurationLSNA\20180201_Scope_Measurements'


In [108]:
table.save()

In [197]:
table.path


'C:\\Users\\sandersa\\PyCharmProjects\\Jupyter-Notebooks\\Scope_Measurement_20180201_003.dat'

In [198]:
table.path=r'C:\Share\LongDurationLSNA\20180201_Scope_Measurements\Scope_Measurement_20180201_005.dat'
table.save()

In [199]:
table.header=[]
for parameter,value in sine.parameter_values.iteritems():
    table.header.append("{0} = {1}".format(parameter,value))

In [200]:
table.options["treat_header_as_comment"]=True

In [201]:
table.options["comment_begin"]="#"
table.options["comment_end"]="\n"

In [202]:
Notes=" Set Channel 3 Frequency from port 3 of ZVA is 3 GHZ Moved "
table.header.append(Notes)

In [195]:
table.save()

In [203]:
Notes=" Set Channel 2 Frequency from port 1 of ZVA is 250/256 * 3 GHZ "
table.header.append(Notes)

In [204]:
table.save()

In [209]:
sine.parameter_values["omega"]/(2*np.pi)/3

11.404426483130687

In [146]:
plt.plot(x_data,y_data)

In [143]:
plt.show()

In [147]:
first_table=table.copy()

In [152]:
oscope.get_timebase_scale()

0.1

In [153]:
oscope.measure_options

{'add_header': False,
 'channels': [1, 2, 3, 4],
 'data_format': 'dat',
 'directory': 'C:\\Users\\sandersa\\PyCharmProjects\\Jupyter-Notebooks',
 'general_descriptor': 'Measurement',
 'initial_time_offset': 26.2,
 'number_frames': 12,
 'number_points': 300,
 'output_table_options': {'data_delimiter': '  ',
  'treat_header_as_comment': True},
 'save_data': False,
 'specific_descriptor': 'Scope',
 'timebase_scale': 0.1,
 'timeout_measurement': 10000,
 'timestep': 0.0003333333333333334}

In [215]:
Channels=["CHAN2","CHAN3","CHAN4"]
for channel in Channels:
    x_data=np.array(map(lambda x:float(x),table["Time"]))
    y_data=np.array(map(lambda x:float(x),table[channel]))
    plt.plot(x_data,y_data,label="{0} Measurement After Move".format(channel))
    plt.plot(x_data,np.array(map(lambda x:float(x),first_table[channel])),label="{0} Measurement Before Move".format(channel))
plt.legend()
plt.show()

In [ ]:
vna=
vna.set_frequency(start=vna_frequency_start,
                  stop=vna_frequency_stop,
                  number_points=vna_frequency_number_points,
                  type=vna_sweep_type)
# Now we set the IFBW of the vna
vna.set_IFBW(vna_IFBW)
# now we set the power in dB
vna.set_power(vna_power)
vna.initialize_w2p()
short_w2p=vna.measure_w2p()

In [212]:
zva=VNA("GPIB::20")

VisaIOError: VI_ERROR_CONN_LOST (-1073807194): The connection for the given session has been lost.